## Setup PyDrake using underactuated in Google Colab

In [19]:
import sys
import importlib
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_underactuated
  setup_underactuated()

## Import Libraries

In [20]:
# Do general python imports

import os
import numpy as np
import matplotlib.pyplot as plt

# Drake Imports

from pydrake.all import (DiagramBuilder, AddMultibodyPlantSceneGraph, Parser, MultibodyPlant, Variable, Simulator)
from pydrake.all import Linearize, LinearQuadraticRegulator
from pydrake.all import plot_system_graphviz, PlanarSceneGraphVisualizer

from underactuated.jupyter import AdvanceToAndVisualize, SetupMatplotlibBackend
plt_is_interactive = SetupMatplotlibBackend()

## Setup URDF Model Path

In [21]:
if 'google.colab' in sys.modules:
  # Check if this cell has already been run:
  if not os.path.isdir('DrakeTests/freeFloating/'):
    !git clone https://github.com/vyas-shubham/DrakeTests.git
  urdfFolderPath = 'DrakeTests/freeFloating/'
else:
  urdfFolderPath = ''

# Comment/Uncomment the lines below to select a S/C URDF
filePath = urdfFolderPath+'planarSC_no_collision.urdf'
assert os.path.isfile(filePath)

## Continuous Time MultiBodyPlant

In [22]:
builder = DiagramBuilder()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
# Note that we parse into both the plant and the scene_graph here.
Parser(plant, scene_graph).AddModelFromFile(filePath)
# Remove gravity
plantGravityField = plant.gravity_field()
plantGravityField.set_gravity_vector([0,0,0])
plant.Finalize()

## Create a Continuous Time Linear System

In [23]:
# Linearize around Zero position and Velocity
joints_default_pos = [0, 0, 0]
joints_default_vel = [0, 0, 0]
# Deafault Free-Floating Base Position (quaternion/position) and velocities (angular/linear)
base_default_pose = [1.0, 0, 0, 0, 0, 0, 0]
base_default_vel = [0, 0, 0, 0, 0, 0]
q = base_default_pose+joints_default_pos
q_d = base_default_vel+joints_default_vel

linearize_context = plant.CreateDefaultContext()
plant.SetPositions(linearize_context, q)
plant.SetVelocities(linearize_context, q_d)

# To linearize about the fixed point, set all torques to zero
jointAcutation =  np.zeros((plant.num_actuators(),1))
actuation_port = plant.get_actuation_input_port()
actuation_port.FixValue(linearize_context, jointAcutation)


# Linearize about this point

plant_linear = Linearize(plant, linearize_context, input_port_index=actuation_port.get_index()
                         ,output_port_index=plant.get_state_output_port().get_index())

AMatrix = plant_linear.A()
BMatrix = plant_linear.B()
CMatrix = plant_linear.C()
DMatrix = plant_linear.D()


diagram = builder.Build()
context = diagram.CreateDefaultContext()

## Manual Linearization and Matrices Comparision

In [24]:
MassMatrix = plant.CalcMassMatrixViaInverseDynamics(linearize_context)
ActuationMatrix = plant.MakeActuationMatrix()

lowerBlockLinearBMatrix = np.linalg.inv(MassMatrix).dot(ActuationMatrix)

In [25]:
print(AMatrix.shape)

(19, 19)


In [17]:
print(BMatrix.shape)

(19, 3)


In [18]:
print(plant.num_positions(), plant.num_velocities())

10 9


In [7]:
print(BMatrix)

[[-0.         -0.         -0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-0.         -0.         -0.        ]
 [-0.03629935  0.02990013 -0.00890808]
 [-0.         -0.         -0.        ]
 [-0.01352109  0.01898231 -0.00565536]
 [-0.         -0.         -0.        ]
 [-0.         -0.         -0.        ]
 [ 0.3387484  -0.65943203  0.39703403]
 [-0.65943203  1.64583791 -1.28297244]
 [ 0.39703403 -1.28297244  1.33511675]]


In [8]:
print(lowerBlockLinearBMatrix)

[[ 0.          0.          0.        ]
 [-0.03629935  0.02990013 -0.00890808]
 [ 0.          0.          0.        ]
 [-0.01352109  0.01898231 -0.00565536]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.3387484  -0.65943203  0.39703403]
 [-0.65943203  1.64583791 -1.28297244]
 [ 0.39703403 -1.28297244  1.33511675]]


## Discrete Time MultiBodyPlant

In [9]:
builder = DiagramBuilder()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
# Note that we parse into both the plant and the scene_graph here.
Parser(plant, scene_graph).AddModelFromFile(filePath)
# Remove gravity
plantGravityField = plant.gravity_field()
plantGravityField.set_gravity_vector([0,0,0])
plant.Finalize()

## Create a Discrete Time Linear System

In [10]:
# Linearize around Zero position and Velocity
joints_default_pos = [0, 0, 0]
joints_default_vel = [0, 0, 0]
# Deafault Free-Floating Base Position (quaternion/position) and velocities (angular/linear)
base_default_pose = [1.0, 0, 0, 0, 0, 0, 0]
base_default_vel = [0, 0, 0, 0, 0, 0]
q = base_default_pose+joints_default_pos
q_d = base_default_vel+joints_default_vel

linearize_context = plant.CreateDefaultContext()
plant.SetPositions(linearize_context, q)
plant.SetVelocities(linearize_context, q_d)

# To linearize about the fixed point, set all torques to zero
jointAcutation =  np.zeros((plant.num_actuators(),1))
actuation_port = plant.get_actuation_input_port()
actuation_port.FixValue(linearize_context, jointAcutation)


# Linearize about this point

plant_linear = Linearize(plant, linearize_context, input_port_index=actuation_port.get_index()
                         ,output_port_index=plant.get_state_output_port().get_index())

AMatrix = plant_linear.A()
BMatrix = plant_linear.B()
CMatrix = plant_linear.C()
DMatrix = plant_linear.D()


diagram = builder.Build()
context = diagram.CreateDefaultContext()

## Manual Linearization and Matrices Comparision

In [11]:
MassMatrix = plant.CalcMassMatrixViaInverseDynamics(linearize_context)
ActuationMatrix = plant.MakeActuationMatrix()

lowerBlockLinearBMatrix = np.linalg.inv(MassMatrix).dot(ActuationMatrix)

In [12]:
print(BMatrix)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.81496765e-08  1.49500662e-08 -4.45404199e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.35210940e-08  1.89823127e-08 -5.65536074e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.38748405e-07 -6.59432026e-07  3.97034035e-07]
 [-6.59432026e-07  1.64583791e-06 -1.28297244e-06]
 [ 3.97034035e-07 -1.28297244e-06  1.33511675e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-3.62993529e-05  2.99001325e-05 -8.90808398e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.35210940e-05  1.89823127e-05 -5.65536074e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.38748405e-04 -6.59432026e-04  3.97034035e-04]
 [-6.59432026e-04  1.64583791e-03 -1.28297244e-03]
 [ 3.97034035e-04 -1.28297244e-03  1.33511675e-03]]


In [13]:
print(lowerBlockLinearBMatrix)

[[ 0.          0.          0.        ]
 [-0.03629935  0.02990013 -0.00890808]
 [ 0.          0.          0.        ]
 [-0.01352109  0.01898231 -0.00565536]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.3387484  -0.65943203  0.39703403]
 [-0.65943203  1.64583791 -1.28297244]
 [ 0.39703403 -1.28297244  1.33511675]]


In [14]:
print(AMatrix)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  5.00000000e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.77555756e-20  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  5.00000000e-04
   0.00000000e+00  2.77555756e-20  0.00000000e+00  0.00000000e+00
   1.81496765e-09 -1.49500662e-09  4.45404199e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00